# Simple Simulation of Alanine Dipeptide
Authors: Benjamin D. Madej & Ross Walker<br>
Original Link: https://ambermd.org/tutorials/basic/tutorial0/index.php<br>
Adapted by: Jeremy Leung<br>
Email:&nbsp;&nbsp; jml230@pitt.edu


## Introduction

This tutorial is designed to provide an introduction to molecular dynamics simulations with Amber. It is designed for AMBER 24 and for new users who want to learn about how to run molecular dynamics simulations with Amber. This notebook is designed with the assumption that you are working within a virtual environment on the H2P Cluster at Pitt.

AMBER stands for Assisted Model Building and Energy Refinement. It refers not only to the molecular dynamics programs, but also a set of force fields that describe the potential energy function and parameters of the interactions of biomolecules.

In order to run a Molecular Dynamics simulation in Amber, each molecule's interactions are described by a molecular force field. The force field has specific parameters defined for each molecule.

<code>sander</code> is the basic MD engine of Amber. <code>pmemd</code> is the high performance implementation of the MD engine that contains a subset of features of <code>sander</code>. <code>pmemd</code> can also be run with acceleration from graphics processing units (GPU) through <code>pmemd.cuda</code> or the MPI parallel version <code>pmemd.mpi</code>.

In order to run an MD simulation with sander or pmemd, three key files are needed:

    parm7 - The file that describes the parameter and topology of the molecules in the system
    rst7 - The file that describes the initial molecular coordinates of the system
    mdin - The file that describes the settings for the Amber MD engine


## Learning Objectives

- navigate the command line interface through terminal and tleap to prepare topology and coordinate files
- understand the basics of forcefields and be able to load the FF19SB forcefield to be able to work with the alanine dipeptide
- set up an explicit water simulation in tleap
- perform basic equilibration
- perform production run simulations at a given temperature
- Visualize the results of production MD in <code>nglview</code>
- Use <code>matplotlib</code> to plot MD thermodynamic data such as temperature, density, and energy
- Use <code>mdtraj</code> to calculate the root mean square displacement (RMSD) of the trajectory relative to the initial structure



## System Requirements
- AmberTools24 is necessary to run this simulation.
    - Note that it is already installed as a module on H2P
- numpy, matplotlib, ipympyl, mdtraj, and nglview are used in this jupyter notebook. 
    - nglview, matplotlib, ipympl, and numpy are optional for visualization purposes.
    - These are already installed as part of the virtual environment.

## Building the Simulation System (Alanine Dipeptide)

We will now begin to build the solvated alanine dipeptide from scratch using a program called `tleap`.

The first line of the following cell is a jupyter notebook "magic command" to export the next lines of the cell into a file called <code>build_diala.in</code>. The rest are commands telling <code>tleap</code> to generate a solvated alanine dipeptide. Expand the next section if you would like to know what each line means!

We can build an alanine dipeptide as an alanine amino acid capped with an acetyl group on the N-terminus and N-methylamide on the C-terminus. After we loaded the force field ff19SB, tLEaP now has these "units" available to build into a molecule. The sequence command will create a new unit from the available units and connect them together.

In [7]:
%%writefile build_diala.in  
source leaprc.protein.ff19SB  
source leaprc.water.opc
diala = sequence { ACE ALA NME } 
solvateoct diala OPCBOX 10.0
saveamberparm diala parm7 rst7
quit

Overwriting build_diala.in


### Concepts

#### Load a protein force field
A force field is the Hamiltonian (potential energy function) and the related parameters that describe the intra- and intermolecular interactions between the molecules in the system. In MD, the Hamiltonian is integrated to describe the forces and velocities of the molecules (See [Allen and Tildesley](https://global.oup.com/academic/product/computer-simulation-of-liquids-9780198803201?cc=us&lang=en&#)).

The basic form of the Amber force field is:

![image](./img/Amber_Hamiltonian.png)
In order to run a molecular dynamics simulation, we need to load a force field to describe the potential energy of alanine dipeptide. We will use the AMBER force field FF19SB1for proteins. To learn more about force fields, please look through section 3.1.1 on page 34 of the [Amber 2024 Manual](https://ambermd.org/doc12/Amber24.pdf#page=34). 

- The first two `source` lines load in force field parameters(for the Amber ff19SB force field and OPC 3-point water model parameters, respectively).
```
source leaprc.protein.ff19SB  
source leaprc.water.opc
```

#### Build alanine dipeptide
We can build an alanine dipeptide as an alanine amino acid capped with an acetyl group on the N-terminus and N-methylamide on the C-terminus. After we loaded the force field ff19SB, tLEaP now has these "units" available to build into a molecule. The sequence command will create a new unit from the available units and connect them together.

- We will use sequence to create a new unit called diala out of the ACE, ALA, and NME units.  ACE, ALA and NME stands for {acetyl group, alanine, N-methyl amide}, respectively. Note that when you connect the acetyl group with the N-methyl amide you get an alanine residue!

```
diala = sequence { ACE ALA NME }
```

#### Solvate alanine dipeptide

The next step to prepare this alanine dipeptide system is to solvate the molecule with explicit water molecules. There are many water models available for MD simulations. In this simulation we will add OPC water molecules2 to the system. The ff19SB force field gives the best performance with the OPC water model and is strongly recommended (see page 36 of the Amber 2020 Manual). To learn more about the OPC water model, please look at section 3.5.1 on pages 53 and 54 of the Amber 2020 Manual.

In this type of simulation, the system has periodic boundary conditions, meaning that molecules that exit one side of the system will wrap to the other side of the system. It is important that the periodic box is large enough, i.e. there is enough water surrounding alanine dipeptide, so that the alanine dipeptide molecule does not interact with its periodic images.

- Solvating the alanine dipeptide into a cubic water box with at least 10 Å clearance to the edge of the box. OPCBOX specifies the type of specifies the type of water box to solvate the solute (diala). The 10.0 indicates that the molecule should have a buffer of at least 10 Angstroms between alanine dipeptide and the periodic box wall.   
```
solvateoct diala OPCBOX  10.0
```

#### Save the Amber parm7 and rst7 input files
Now we will save the parm7 and rst7 files to the current working directory. The parm7 topology file defines which atoms are bonded to each other and the rst7 coordinate file defines where each atom is located on a 3-dimensional coordinate plane. The unit diala now contains the alanine dipeptide molecule, water molecules, and the periodic box information necessary for simulation. The parameters will be assigned from the ff19SB force field. 


- Export the peptide into a paramter file `parm7` and a coordinate file `rst7`.
```
saveamberparm diala parm7 rst7
```


#### Quit tLEaP

- Exit from the `tleap` program at the end
```
quit
```

### Running `tleap`

Run `tleap` with the following line. The first half loads in the Amber24 module from the cluster. The second half (`tleap ...`) is the actually command we will run.

If you see the following lines at the end of the output, you've successfully built the system!
```
Exiting LEaP: Errors = 0; Warnings = 0; Notes = 1.
```

In [6]:
!module load gcc/10.2.0 openmpi/4.1.1 amber/24 &&\
tleap -f build_diala.in 

         -gencode arch=compute_61,code=[compute_61,sm_61] 
         -gencode arch=compute_70,code=[compute_70,sm_70] 
         -gencode arch=compute_89,code=[compute_89,sm_89] 
If running on TEACH cluster:         -arch=sm_52 
-I: Adding /ihome/crc/install/amber/amber24/amber24/dat/leap/prep to search path.
-I: Adding /ihome/crc/install/amber/amber24/amber24/dat/leap/lib to search path.
-I: Adding /ihome/crc/install/amber/amber24/amber24/dat/leap/parm to search path.
-I: Adding /ihome/crc/install/amber/amber24/amber24/dat/leap/cmd to search path.
-f: Source build_diala.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./build_diala.in
----- Source: /ihome/crc/install/amber/amber24/amber24/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /ihome/crc/install/amber/amber24/amber24/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /ihome/crc/install/amber/amber24/amber24/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc

## Prepare Amber MD input files

## Running the Simulation

The files to initialize and run the WESTPA simulation are included with this jupyter notebook for demonstration purposes. **It is <u>not</u> recommended that you run your WE simulations within a Jupyter Notebook.** The simulation will take a while to complete, so feel free to stop it at any stage. Sample completed files for analysis are provided in the `for_analysis/` directory and `w_crawl/` folder.

In [ ]:
import os
import shutil
# Clean up from previous/ failed simulations.
for i in ['west.h5', 'seg_logs', 'traj_segs','istates','get_pcoord.log']:
    try:
        os.remove(i)
    except OSError:
        try:
            shutil.rmtree(i)
        except OSError:
            pass
        
for i in ['seg_logs','traj_segs','istates']:
    os.mkdir(i)

In [ ]:
import westpa
import numpy
from westpa.cli.core import w_init
from argparse import Namespace

# Initializing the System:
# Set some parameters that WESTPA needs to set simulation state.
args = Namespace(rcfile='west.cfg',
                 verbosity='verbose', ## change to 'debug' if you want a more detailed view of what's happening
                                      ## or 'verbose' for some information
                                      ## or 'quiet' for no information at all. 
                 work_manager='threads')

# Update westpa.rc with these
westpa.rc.process_args(args)

# Initialize the simulation using the tstate and bstate files
w_init.initialize(tstates=None, bstates=None, 
                  tstate_file='tstate.file', bstate_file='bstates/bstates.txt', 
                  segs_per_state=5, shotgun=False)

In [ ]:
import westpa
import numpy
from westpa.cli.core import w_run
import westpa.work_managers as work_managers
from argparse import Namespace

# Running the Simulation.
# Set some parameters that WESTPA needs to set simulation state.
args = Namespace(rcfile = 'west.cfg',
                 verbosity = "verbose",
                 work_manager = 'threads')

# Update westpa.rc with these
westpa.rc.process_args(args)
# Prepare work manager
work_managers.environment.process_wm_args(args)

# Launch the simulation
w_run.run_simulation()

## Monitoring the WE Simulation

The following sample code runs <code>w_pdist</code> and <code>plothist</code> to generate a probability distribution.

In [ ]:
import tarfile

# Untar the files for analysis
for file in ['./for_analysis/01.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')

In [ ]:
# Generate the pdist.h5
print('running w_pdist...')
!{'w_pdist -W ./for_analysis/01/west.h5'}
print('Done!')

In [ ]:
# Generate the average hist.pdf
print('running plothist...')
!{'plothist evolution -o evol.pdf pdist.h5'}
print('Done!')

In [ ]:
# View the average probability distribution
from IPython.display import IFrame, display
filepath = "evol.pdf"
IFrame(filepath, width=700, height=400)

In [ ]:
# Generate the average hist.pdf
print('running plothist...')
!{'plothist average -o avg.pdf pdist.h5 0'}
print('Done!')

In [ ]:
# View the average probability distribution
from IPython.display import IFrame, display
filepath = "avg.pdf"
IFrame(filepath, width=700, height=400)

## Analyzing the WE Simulation

### Visualization of the System

We will now take a look at how one of the basis states looks like. The water box is omitted for visibility. Na<sup>+</sup> is red and Cl<sup>-</sup> is blue in the representation.

In [ ]:
import mdtraj
import nglview
system = mdtraj.load('bstates/01/basis.xml', top='common_files/bstate.pdb')
Na = list(range(0,1)) # Na+ is the first atom
Cl = list(range(1,2)) # Cl- is the second atom
both = Na + Cl # Na+ + Cl-
system = system.atom_slice(both)
view = nglview.show_mdtraj(system)
view.representations = [
    {"type": "ball+stick", "params": {
        "sele": ".Na+", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": ".Cl-", "color": "blue"
    }}
]
view.background = 'white'
view

### Visualization of Coverage of the Start States

The following cells plots the multiple start states in a way that allows us to examine  their spacial coverage. The first cell visualizes the system one-by-one after alignment to Na<sup>+</sup>. The second cell plots all the center-of-masses at once. In some frames, Cl<sup>-</sup> might not be visible unless you move the camera.

Do note that many lines are not necessary for Na<sup>+</sup>/Cl</sup>-</sup> as it is spherically symmetrical, and that its center of mass is equal to its coordinate. Extra code is provided so it could be generalized to larger systems.

In [ ]:
import numpy
import mdtraj
import nglview
lst = numpy.loadtxt('bstates/bstates.txt', usecols=2, dtype=str) # Reading basis state names
tpg = 'common_files/bstate.pdb' # Topology File for basis state (Shared between all bstates)
lst = [x + "/basis.xml" for x in lst] # Change path to point to file name

# Reading reference and setup
# There might be some warnings about unconverged rotation matrices because of the system's rotational symmetry

com = [] # list containing all CoM
a = mdtraj.load('bstates/01/basis.xml', top=tpg) # Load the first
a_slice = a.atom_slice([0,1]) # Just Na+
com.append(numpy.squeeze(mdtraj.compute_center_of_mass(a_slice))) # Save CoM of Na+
a_slice = a.atom_slice([0,1]) # Both Na+/Cl-

# Loading and superposing, storing Center of Mass (CoM) to list for heatmap
c = a_slice
for i in lst:
    b = mdtraj.load('bstates/' + i, top=tpg)
    b = b.atom_slice([0,1])
    b.superpose(a_slice, atom_indices=[0])
    c = mdtraj.join([c,b], check_topology=False)
    # Just saving the CoM of Cl-, since Na+ is superimposed
    com.append(numpy.squeeze(mdtraj.compute_center_of_mass(b.atom_slice([1]))))
com = numpy.asarray(com)

# Now displaying it, note that Cl- is not visible in some frames unless you rotate the camera
view2 = nglview.show_mdtraj(c)
view2.representations = [
    {"type": "ball+stick", "params": {
        "sele": ".Na+", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": ".Cl-", "color": "blue"
    }}
]
view2.center('.Na+')
view2.control.zoom(-1.75)
view2

In [ ]:
# Looking at the coverage of the sstates, assuming you ran the previous cell
# Comment out the next line or install ipypml if you have trouble viewing.
%matplotlib widget 
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import numpy

fig = matplotlib.pyplot.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

img1 = ax.scatter(com[0,0], com[0,1], com[0,2], s=20, marker='s', color='Red', label='Na+')
img2 = ax.scatter(com[1:,0], com[1:,1], com[1:,2], s=50, marker='.', color='Blue', label='Cl-')

# Labels and Titles
ax.set_title("Basis State Structures Coverage")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.legend()

show()

Due to the rotational symmetry, it might be better to look at the Na<sup>+</sup>/Cl<sup>-</sup> atom-to-atom distances instead, which are already precalculated in each pcoord.init file.

In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
lst = numpy.loadtxt('bstates/bstates.txt', usecols=2, dtype=str) # Reading basis state names
lst2 = ["bstates/" + x + "/pcoord.init" for x in lst] # Change path to point to file name

values = numpy.asarray([numpy.loadtxt(x) for x in lst2])

fig2 = matplotlib.pyplot.figure(figsize=(7,7))
ax2 = fig2.subplots()

img3 = ax2.scatter(0, 0, color='Red', label='Na+')
img4 = ax2.scatter(values[:], numpy.zeros(values.shape[0]), color='Blue', label='Cl-')
img5 = ax2.axvline(x=2.6, ymin=0, ymax=1, label='Bound State', linestyle='--')

# Labels and Titles
ax2.set_title("Basis States Structures Coverage (Atom-to-Atom Distances)")
ax2.set_xlabel('Na+ to Cl- distance ($\mathrm{\AA}$)')
ax2.set_ylabel('y-axis')
ax2.legend()
plt.xlim(-1,23)

plt.show()

### Calculating Rates

In [ ]:
import tarfile
from os import symlink

# Untar the files for analysis.
for file in ['./for_analysis/01.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')
try:
    symlink('./for_analysis/01/west.h5', './west.h5')
except FileExistsError:
    print('A different west.h5 exists in the basic_nacl/. Either delete it or rename it to continue.')

In [ ]:
from westpa.cli.tools import w_ipa
import westpa

w = w_ipa.WIPI()
w.main()
w.interface = 'matplotlib'

## Managing Your Simulations

### Combining Multiple Simulation Runs

The following sample code runs <code>w_multi_west</code> to concatenate two runs.

In [ ]:
import tarfile

# Untar the files
for file in ['./for_analysis/01.tar.gz','./for_analysis/02.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')

In [ ]:
# Run w_multi_west in the commandline
!{'w_multi_west -m ./for_analysis/ -n 2'}
print('Done!')

In [ ]:
# Check to see if the multi.h5 file exists
from os.path import exists
exists('multi.h5')

### Using <code>w_crawl</code> to calculate post-simulation auxiliary data

The following sample code runs <code>w_crawl</code> to calculate additional observables post-simulation.

In [ ]:
import os
os.chdir('w_crawl')
# Run w_crawl in the commandline
!{'./run_w_crawl.sh'}
os.chdir('../')
print('Done!')

In [ ]:
# Check to see if the example.h5 file exists
from os.path import exists
exists('./w_crawl/crawl.h5')

## Cleaning Up

In [ ]:
# Run the following bash script to revert your tutorial folder to pristine condition.
!{'./1.clean.sh'}